In [16]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "C0de1sFun"

# Connect to database via CRUD Module
db = AnimalShelter()

# Class read method must support return of list object and accept projection json input
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    dcc.Store(id='filtered-data', storage_type='memory'),  # Store for filtered data
    html.Div(
        html.A(
            html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={'width': '20%', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}
            ),
            href="https://www.snhu.edu"
        ),
        style={'textAlign': 'center', 'padding': '20px'}
    ),
    html.Div(id='hidden-div', style={'display': 'none'}),
    html.Center(html.B(html.H1('Rylan Champion 2024  CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'all'}
            ],
            value='all',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        row_selectable='single',
        selected_rows=[0],
        style_cell={'textAlign': 'left'},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
        style_data={'whiteSpace': 'normal', 'height': 'auto'}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('filtered-data', 'data'),
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if df.empty:
        return [], []
    
    filtered_df = df
    
    if filter_type == 'water':
        filtered_df = df[
            (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
            (df['sex_upon_outcome'] == 'Intact Female') &
            (df['age_upon_outcome_in_weeks'] <= 156)&
            (df['age_upon_outcome_in_weeks'] >= 26)
            
        ]
    elif filter_type == 'mountain':
        filtered_df = df[
            (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] <= 156)&
            (df['age_upon_outcome_in_weeks'] >= 26)
        ]
    elif filter_type == 'disaster':
        filtered_df = df[
            (df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
            (df['sex_upon_outcome'] == 'Intact Male') &
            (df['age_upon_outcome_in_weeks'] <= 312)&
            (df['age_upon_outcome_in_weeks'] >= 20)
        ]
    
    return filtered_df.to_dict('records'), filtered_df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('filtered-data', 'data')]
)
def update_graphs(viewData):
    if viewData:
        filtered_df = pd.DataFrame(viewData)
        if not filtered_df.empty:
            fig = px.pie(filtered_df, names='breed', title='Preferred Animals')
            return [dcc.Graph(figure=fig)]
    return [dcc.Graph(figure=px.pie(pd.DataFrame(), names=None, title='No Data Available'))]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(filtered_data, selected_rows):
    if filtered_data:
        filtered_df = pd.DataFrame(filtered_data)
        if selected_rows:
            row = selected_rows[0]
        else:
            row = 0  # Default to first row if nothing selected
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[filtered_df.iloc[row, 13], filtered_df.iloc[row, 14]],  # Adjust indices based on your data
                          children=[
                              dl.Tooltip(filtered_df.iloc[row, 4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(filtered_df.iloc[row, 9])
                              ])
                          ])
            ])
        ]
    return [dl.Map(style={'width': '500px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()])]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:26589/
